In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_squared_error
import os
import sys
import math
import seaborn as sns

import analysis.model_assessment
import analysis.train_data_analysis
from analysis.model_assessment import *
from analysis.per_scene_scatter import *
from scipy.stats import linregress

In [37]:
data = np.load('../munged_1285_cloud_mask.npz')
refls = data['output_rfl']
wv = data['output_wv']
fids = data['output_idx']
cloud_mask = data['output_cloud_flags']



values, counts = np.unique(fids, return_counts=True)
scenes = np.split(refls, 1285)
wv_by_scene = np.split(wv, 1285)
scenes = np.array(scenes)
wv_by_scene = np.array(wv_by_scene)

print(scenes.shape)
print(wv_by_scene.shape)

# Getting rid of unrealistic crosstrack 
bad = []
for i in range(wv_by_scene.shape[0]):
    scene = wv_by_scene[i].copy()
    if scene[np.where((scene > 6) | (scene < 0))].shape[0] != 0:
        bad.append(i)

wv_by_scene = np.delete(wv_by_scene, bad, axis = 0)
scenes = np.delete(scenes, bad, axis = 0)
cloud_mask = np.delete(cloud_mask, bad, axis = 0)

print(scenes.shape)
print(wv_by_scene.shape)
print(cloud_mask.shape)

scenes_resh = scenes.reshape((scenes.shape[0]*scenes.shape[1], scenes.shape[2]))
cloud_mask_resh = cloud_mask.flatten()
wv_resh = wv_by_scene.flatten()
scenes_output = np.zeros(scenes.shape)
scenes_output[:] = np.nan
wv_output = np.zeros(wv_by_scene.shape)
wv_output[:] = np.nan

print('scenes output: ', scenes_output.shape)

scenes_resh = scenes_resh[cloud_mask_resh == 0, :]
wv_by_scene = wv_by_scene[cloud_mask == 0]

scenes_output[cloud_mask == 0, :] = scenes_resh
wv_output[cloud_mask == 0] = wv_by_scene

print('wv output: ', wv_output.shape)

print('scenes reshape: ', scenes_resh.shape) # goes to the model
print('wv: ' , wv_by_scene.flatten().shape) # labeled data for the model and test set



(1285, 1242, 285)
(1285, 1242)
(1254, 1242, 285)
(1254, 1242)
(1254, 1242)
scenes output:  (1254, 1242, 285)
wv output:  (1254, 1242)
scenes reshape:  (1301254, 285)
wv:  (1301254,)


In [38]:
cloud_mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [39]:
wv_output

array([[1.50906026, 1.50907362, 1.50909722, ..., 1.66015291, 1.66326976,
        1.66501236],
       [1.78841186, 1.78867328, 1.78855801, ..., 1.78433836, 1.78593075,
        1.78687382],
       [1.76582718, 1.76552701, 1.76496899, ..., 1.80048525, 1.80299723,
        1.8045243 ],
       ...,
       [5.84847593, 5.84667063, 5.84392118, ..., 3.18618727, 3.21626163,
        3.23325849],
       [       nan,        nan,        nan, ..., 5.69170713, 5.72420835,
        5.742136  ],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [40]:
scenes_output_reshaped[:,:,0]

array([[0.20281757, 0.20277312, 0.20265527, ..., 0.21301824, 0.21221894,
        0.21221669],
       [0.20662118, 0.20645121, 0.20692374, ..., 0.20872357, 0.20946708,
        0.20953585],
       [0.21582541, 0.21606272, 0.214524  , ..., 0.21342817, 0.21336432,
        0.21248835],
       ...,
       [0.20250189, 0.20654593, 0.20588894, ..., 0.23707002, 0.23967285,
        0.25061804],
       [       nan,        nan,        nan, ..., 0.18614188, 0.18517491,
        0.18838981],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])